<a href="https://colab.research.google.com/github/galbendavids/galbendavids/blob/main/dicta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.37.2
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 101

In [2]:
import transformers
transformers.__version__

'4.37.2'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "dicta-il/dictalm-7b-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_megatron_gpt.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dicta-il/dictalm-7b-instruct:
- configuration_megatron_gpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_megatron_gpt.py:   0%|          | 0.00/55.1k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/950M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
#import pandas as pd
#file_path = 'https://www.dropbox.com/scl/fi/450wrvtp3yxk6oe2bwnfs/3_articles.csv?rlkey=ze97161x6auiwf8hh0rvcdqnc&dl=1'
#df = pd.read_csv(file_path)
#df.head()

,text
0,"המסע של לילי גלדסטון, ""הנערה מהשמורה"", עוד יסת..."
1,עוגיות קשיו עדינות למה לא משלבים מספיק קשיו בא...
2,"תבשיל עוף הציידים של טוסקנה תבשיל עוף, יין אדו..."


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('dicta-il/dictalm-7b-instruct')
# model = AutoModelForCausalLM.from_pretrained('dicta-il/dictalm-7b-instruct', trust_remote_code=True, load_in_8bit=True).cuda()

model.eval()

with torch.inference_mode():
  #prompt = df.loc[1, 'text']
  prompt =
  "
  אתה הצ׳טבוט של חברת ישראכרט.
אתה עוזר לאורית פינצ׳י בכל שאלה שהיא תצטרך לגבי ההלוואה שלה.
הינה הפרטים
שם החברה : אורית השכרת רכבים בע״מ
סכום ההלוואה:  300,000 ש״ח
ריבית : 4.1 אחוז
תאריך החזרת ההלוואה: 30.12.2027
גרייס: ללא חודשי גרייס
סטטוס בקשת ההלוואה: באישור ראשוני
איסוף הסכמה: עדיין לא
איסוף מסמכים: עדיין לא


ידע כללי שצריך להיות לך על מדיניות ישראכרט:
ריבית על הלוואה הינה 4.5 אחוז החל מ350,001 ש״ח
ריבית על הלוואה הינה 4.1 אחוז עד 350,000 ש״ח
במקרים פרטניים תאושר ריבית נמוכה יותר אך התהליך יקח יותר זמן.

כל לקוח נדרש לאשר הסכמה על דו״ח אשראי בקישור שישלח אליו,
כל לקוח נדרש להעלות מסמכים בקישור שישלח אליו

________
בקשה חדשה:
שלום
  "
  kwargs = dict(
        inputs=tokenizer(prompt, return_tensors='pt').input_ids.to(model.device),
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.1,
        max_length=2000,
        min_new_tokens=5
    )

  #print(tokenizer.batch_decode(model.generate(**kwargs), skip_special_tokens=True))
  print(''.join(tokenizer.batch_decode(model.generate(**kwargs), skip_special_tokens=True)).replace(prompt,""))

SyntaxError: unterminated string literal (detected at line 12) (<ipython-input-14-2786472cc091>, line 12)